In [2]:
# def remove_precursor_peak(df):
#     #if there is a peak that is 10X more intense than everything else, get rid of it (it's a precursor that hasn't been fragmented)
#     maxpeak = df['intensity'].max()
    
#     #if it's ten times more intesne that all the other peaks get rid of it. 
#     rm_max_peak = df[df['intensity']!=maxpeak]
#     mean_intensity = rm_max_peak['intensity'].nlargest(10).mean()
    
#     if maxpeak >= 5*mean_intensity:
        
#         #also remove the plusone isotope
#         maxmz = df[df['intensity'] == maxpeak]['mz'].values[0]
#         plusone = maxmz + 1.0034
#         plusone_tol_max = plusone+.01
#         plusone_tol_min = plusone-.01
        
#         plusonemz_array = rm_max_peak[(rm_max_peak['mz'] <= plusone_tol_max) & (rm_max_peak['mz'] >= plusone_tol_min)]['mz'].values
#         if len(plusonemz_array > 0 ): #if there is a plus1 isotope
#             plusonemz = plusonemz_array[0]
#             rm_max_peak = rm_max_peak[rm_max_peak['mz']!=plusonemz]
        
#         return rm_max_peak
    
#     return df

In [2]:
# #Scan number is how you conenct the psm to the mzml so we can tell if something is high or low
# # highest peak in the mzml is 100% intensity

# # SAMPLER - y1 would be just R

# This cell is for testing on an individual file

# file_name = "sc_rep1"


# # PSM
# psm_path = psm_files.get(file_name)

# psm = pd.read_csv(psm_path, sep = '\t', dtype={'Base Sequence': str, 'Missed Cleavages': str, 'Peptide Monoisotopic Mass':str, 'Mass Diff (Da)':str,'Mass Diff (ppm)':str,'	Protein Accession':str,'Peptide Description':str, 'Notch':str, 'Num Variable Mods':str, 'Decoy': str})

# #drop columns that we don't use
# psm = psm[['Matched Ion Series', 'Matched Ion Mass-To-Charge Ratios', 'Matched Ion Intensities', 'Matched Ion Counts', 'QValue', 'Full Sequence', 'Previous Amino Acid', 'Next Amino Acid', 'File Name', 'Scan Number']]
# psm = psm[psm['QValue'] < 0.01]

# psm = psm.rename({"Scan Number": "scan", "Full Sequence": "peptide"}, axis=1)

# #remove duplicate scans
# psm = psm.sort_values("QValue")
# psm = psm.drop_duplicates(subset=["scan"], keep="first")
# #make sure dtypes are correct
# psm[["scan"]] = psm[["scan"]].apply(pd.to_numeric)

# # MZML
# mzml_path = mzml_files.get(file_name)

# mzml = pyteomics.mzml.MzML(mzml_path)
# psm['Base Peak Intensity'] = psm.apply(lambda row: _mzml_helper(row, mzml, "base_peak_intensity"),axis=1)
# psm['mzml_info'] = psm.apply(lambda row: _mzml_helper(row, mzml),axis=1)
# psm[['mz_array', 'intensity_array', 'precursor_intenisty']] = pd.DataFrame(psm['mzml_info'].tolist(), index= psm.index)
# psm=psm.drop(columns='mzml_info')


# # import pandas as pd
# # with pd.option_context('display.max_rows', 15, 'display.max_columns', None):
# #     display(psm)
# psm

In [3]:
def _mzml_helper(row, mzml, data_type="all"):
    scan = str(row['scan'])
    my_id = 'controllerType=0 controllerNumber=1 scan='+ str(scan)
    spectrum_dict = mzml.get_by_id(my_id)

    spectrum_id = spectrum_dict['id']
    retention_time = (spectrum_dict['scanList']['scan'][0].get('scan start time', -1))
    mz_array = list(spectrum_dict['m/z array'])
    intensity_array = list(spectrum_dict['intensity array'])
    
    #base peak intensity
    bpi = spectrum_dict['base peak intensity']
    if (data_type == "base_peak_intensity"):
        return bpi

    #precursor information
    precursor = spectrum_dict['precursorList']['precursor'][0]
    precursor_ion = precursor['selectedIonList']['selectedIon'][0]
    precursor_mz = precursor_ion['selected ion m/z']
    if 'peak intensity' in precursor_ion:
        precursor_intenisty =  precursor_ion['peak intensity']
    else:
        precursor_intenisty = None
    if 'charge state' in precursor_ion:
        precursor_charge = int(precursor_ion['charge state'])
    elif 'possible charge state' in precursor_ion:
        precursor_charge = int(precursor_ion['possible charge state'])
    else:
        precursor_charge = 'NAN'

    all_info = [bpi,mz_array,intensity_array,precursor_intenisty]

    return all_info

In [4]:
import pandas as pd
import numpy as np
import pyteomics.mzml


list_of_all_filenames = [ 'bulk_rep1',"bulk_rep2","bulk_rep3",
                        '2ng_rep1','2ng_rep2','2ng_rep3','2ng_rep4','2ng_rep5','2ng_rep6',
                        '0.2ng_rep1','0.2ng_rep2','0.2ng_rep3','0.2ng_rep4','0.2ng_rep5','0.2ng_rep6',
                        'sc_rep1','sc_rep2','sc_rep3','sc_rep4','sc_rep5']

psm_files = {}
#bulk
psm_files["bulk_rep1"] = "data/psm/Project_PXD011163/OR11_20160122_PG_HeLa_CVB3_CT_A-calib_PSMs.psmtsv.gz"
psm_files["bulk_rep2"] = "data/psm/Project_PXD011163/OR11_20160122_PG_HeLa_CVB3_CT_B-calib_PSMs.psmtsv.gz"
psm_files["bulk_rep3"] = "data/psm/Project_PXD011163/OR11_20160122_PG_HeLa_CVB3_CT_C-calib_PSMs.psmtsv.gz"

#2ng
psm_files["2ng_rep1"] = "data/psm/2ng/Ex_Auto_J3_30umTB_2ngQC_60m_1-calib_PSMs.psmtsv.gz"
psm_files["2ng_rep2"] = "data/psm/2ng/Ex_Auto_J3_30umTB_2ngQC_60m_2-calib_PSMs.psmtsv.gz"
psm_files["2ng_rep3"] = "data/psm/2ng/Ex_Auto_K13_30umTA_2ngQC_60m_1-calib_PSMs.psmtsv.gz"
psm_files["2ng_rep4"] = "data/psm/2ng/Ex_Auto_K13_30umTA_2ngQC_60m_2-calib_PSMs.psmtsv.gz"
psm_files["2ng_rep5"] = "data/psm/2ng/Ex_Auto_W17_30umTB_2ngQC_60m_1-calib_PSMs.psmtsv.gz"
psm_files["2ng_rep6"] = "data/psm/2ng/Ex_Auto_W17_30umTB_2ngQC_60m_2-calib_PSMs.psmtsv.gz"

#.2ng
psm_files["0.2ng_rep1"] = "data/psm/02ng/Ex_Auto_J3_30umTB_02ngQC_60m_1-calib_PSMs.psmtsv.gz"
psm_files["0.2ng_rep2"] = "data/psm/02ng/Ex_Auto_J3_30umTB_02ngQC_60m_2-calib_PSMs.psmtsv.gz"
psm_files["0.2ng_rep3"] = "data/psm/02ng/Ex_Auto_K13_30umTA_02ngQC_60m_1-calib_PSMs.psmtsv.gz"
psm_files["0.2ng_rep4"] = "data/psm/02ng/Ex_Auto_K13_30umTA_02ngQC_60m_2-calib_PSMs.psmtsv.gz"
psm_files["0.2ng_rep5"] = "data/psm/02ng/Ex_Auto_W17_30umTA_02ngQC_60m_3-calib_PSMs.psmtsv.gz"
psm_files["0.2ng_rep6"] = "data/psm/02ng/Ex_Auto_W17_30umTA_02ngQC_60m_4-calib_PSMs.psmtsv.gz"
#
#True SC
psm_files["sc_rep1"] = "data/psm/true_SC/D19_15um30cm_SC1-calib_PSMs.psmtsv.gz"
psm_files["sc_rep2"] = "data/psm/true_SC/D19_15um30cm_SC2-calib_PSMs.psmtsv.gz"
psm_files["sc_rep3"] = "data/psm/true_SC/D19_15um30cm_SC3-calib_PSMs.psmtsv.gz"
psm_files["sc_rep4"] = "data/psm/true_SC/D19_15um30cm_SC4-calib_PSMs.psmtsv.gz"
psm_files["sc_rep5"] = "data/psm/true_SC/D19_15um30cm_SC5-calib_PSMs.psmtsv.gz"

mzml_files = {}
#bulk
mzml_files["bulk_rep1"] = "data/mzMLs/OR11_20160122_PG_HeLa_CVB3_CT_A.mzML"
mzml_files["bulk_rep2"] = "data/mzMLs/OR11_20160122_PG_HeLa_CVB3_CT_B.mzML"
mzml_files["bulk_rep3"] = "data/mzMLs/OR11_20160122_PG_HeLa_CVB3_CT_C.mzML"

#2ng
mzml_files["2ng_rep1"] = "data/mzMLs/Ex_Auto_J3_30umTB_2ngQC_60m_1.mzML"
mzml_files["2ng_rep2"] = "data/mzMLs/Ex_Auto_J3_30umTB_2ngQC_60m_2.mzML"
mzml_files["2ng_rep3"] = "data/mzMLs/Ex_Auto_K13_30umTA_2ngQC_60m_1.mzML"
mzml_files["2ng_rep4"] = "data/mzMLs/Ex_Auto_K13_30umTA_2ngQC_60m_2.mzML"
mzml_files["2ng_rep5"] = "data/mzMLs/Ex_Auto_W17_30umTB_2ngQC_60m_1.mzML"
mzml_files["2ng_rep6"] = "data/mzMLs/Ex_Auto_W17_30umTB_2ngQC_60m_2.mzML"

mzml_files["0.2ng_rep1"] = "data/mzMLs/Ex_Auto_J3_30umTB_02ngQC_60m_1.mzML"
mzml_files["0.2ng_rep2"] = "data/mzMLs/Ex_Auto_J3_30umTB_02ngQC_60m_2.mzML"
mzml_files["0.2ng_rep3"] = "data/mzMLs/Ex_Auto_K13_30umTA_02ngQC_60m_1.mzML"
mzml_files["0.2ng_rep4"] = "data/mzMLs/Ex_Auto_K13_30umTA_02ngQC_60m_2.mzML"
mzml_files["0.2ng_rep5"] = "data/mzMLs/Ex_Auto_W17_30umTA_02ngQC_60m_3.mzML"
mzml_files["0.2ng_rep6"] = "data/mzMLs/Ex_Auto_W17_30umTA_02ngQC_60m_4.mzML"

mzml_files["sc_rep1"] = "data/mzMLs/D19_15um30cm_SC1.mzML"
mzml_files["sc_rep2"] = "data/mzMLs/D19_15um30cm_SC2.mzML"
mzml_files["sc_rep3"] = "data/mzMLs/D19_15um30cm_SC3.mzML"
mzml_files["sc_rep4"] = "data/mzMLs/D19_15um30cm_SC4.mzML"
mzml_files["sc_rep5"] = "data/mzMLs/D19_15um30cm_SC5.mzML"

list_of_sc_filenames = ['sc_rep1','sc_rep2','sc_rep3','sc_rep4','sc_rep5']
list_of_2ng_filenames = ['2ng_rep1', '2ng_rep2', '2ng_rep3', '2ng_rep4', '2ng_rep5', '2ng_rep6']
list_of_02ng_filenames = ['0.2ng_rep1','0.2ng_rep2','0.2ng_rep3','0.2ng_rep4','0.2ng_rep5','0.2ng_rep6']
list_of_filenames_to_parse = ['2ng_rep1','2ng_rep2','2ng_rep3','2ng_rep4','2ng_rep5','2ng_rep6',
                        '0.2ng_rep1','0.2ng_rep2','0.2ng_rep3','0.2ng_rep4','0.2ng_rep5','0.2ng_rep6',
                        'sc_rep1','sc_rep2','sc_rep3','sc_rep4','sc_rep5']

,Matched Ion Series,Matched Ion Mass-To-Charge Ratios,Matched Ion Intensities,Matched Ion Counts,QValue,peptide,Previous Amino Acid,Next Amino Acid,File Name,scan,Base Peak Intensity,mz_array,intensity_array,precursor_intenisty
0,"[b1+1, b2+1, b4+1, b5+1, b6+1, b7+1, b8+1, b9+...","[b1+1:157.10833, b2+1:228.14534, b4+1:412.2303...","[b1+1:372, b2+1:335, b4+1:928, b5+1:2377, b6+1...",23,0.000000,RAPSVANVGSHC[Common Fixed:Carbamidomethyl on C...,R,I,D19_15um30cm_SC1-calib,13311,4185.303711,"[110.07178497314453, 129.10287475585938, 130.0...","[1642.150146484375, 2533.017578125, 1855.31433...",27011.201172
909,"[y1+1, y5+1, y6+1, y7+1, y8+1, y9+1, y10+1, y1...","[y1+1:175.11869, y5+1:463.22661, y6+1:520.2472...","[y1+1:324, y5+1:443, y6+1:1287, y7+1:980, y8+1...",10,0.000000,FSSSSGYGGGSSR,R,V,D19_15um30cm_SC1-calib,9539,2430.047852,"[110.07194519042969, 120.0814437866211, 129.10...","[295.7001647949219, 2430.0478515625, 451.53912...",8357.301758
908,"[y1+1, y2+1, y4+1, y5+1, y7+1, y9+1, y10+1, y1...","[y1+1:147.11271, y2+1:218.14988, y4+1:460.2749...","[y1+1:487, y2+1:299, y4+1:238, y5+1:285, y7+1:...",10,0.000000,LVPGGGATEIELAK,R,Q,D19_15um30cm_SC1-calib,16309,3371.135986,"[115.09736633300781, 120.08153533935547, 122.6...","[213.128173828125, 314.9737854003906, 213.1428...",9776.433594
907,"[y1+1, y2+1, y3+1, y4+1, y5+1, y6+1, y7+1, y8+...","[y1+1:175.11896, y2+1:288.20311, y3+1:403.2289...","[y1+1:867, y2+1:3078, y3+1:300, y4+1:271, y5+1...",10,0.000000,FPGQLNADLR,R,K,D19_15um30cm_SC1-calib,16465,3353.072998,"[110.0720443725586, 118.19334411621094, 120.08...","[206.6062774658203, 226.9688262939453, 1606.02...",10601.946289
906,"[y1+1, y3+1, y4+1, y5+1, y6+1, y7+1, y8+1];[b2...","[y1+1:175.11898, y3+1:391.19291, y4+1:528.2528...","[y1+1:700, y3+1:439, y4+1:1207, y5+1:1680, y6+...",10,0.000000,ALIVLAHSER,R,T,D19_15um30cm_SC1-calib,13602,3838.482178,"[110.07189178466797, 112.99938201904297, 117.4...","[539.782958984375, 250.33363342285156, 278.596...",19559.812500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3240,"[b2+1, b3+1];[y2+1, y8+1, y9+1, y10+1]","[b2+1:199.10743, b3+1:270.14453];[y2+1:274.187...","[b2+1:372, b3+1:355];[y2+1:624, y8+1:553, y9+1...",6,0.009988,TPAFAESVTEGDVR,K,W,D19_15um30cm_SC1-calib,16115,826.098511,"[113.7320785522461, 120.08138275146484, 123.20...","[280.89892578125, 826.0985107421875, 244.73976...",43545.980469
3239,"[b2+1];[y4+1, y5+1, y6+1, y7+1, y8+1]","[b2+1:216.09792];[y4+1:476.28290, y5+1:604.341...","[b2+1:431];[y4+1:377, y5+1:1085, y6+1:1938, y7...",6,0.009988,TNPEIQSTLR,K,K,D19_15um30cm_SC1-calib,12357,2855.330078,"[110.07185363769531, 115.46809387207031, 115.9...","[394.66351318359375, 248.77609252929688, 236.9...",8512.204102
3238,"[y1+1, y2+1, y6+1, y9+1];[b3+1, b10+1]","[y1+1:175.11901, y2+1:288.20331, y6+1:784.3658...","[y1+1:426, y2+1:452, y6+1:405, y9+1:638];[b3+1...",6,0.009988,DSLLQDGEFSM[Common Variable:Oxidation on M]DLR,R,T,D19_15um30cm_SC1-calib,21034,1106.711548,"[123.08111572265625, 136.0764923095703, 138.25...","[271.2136535644531, 356.3466491699219, 245.141...",10259.285156
3241,"[y1+1, y4+1, y5+1, y6+1, y7+1];[b2+1]","[y1+1:175.11868, y4+1:544.32147, y5+1:707.3818...","[y1+1:310, y4+1:278, y5+1:366, y6+1:1143, y7+1...",6,0.009988,DQVTYQQLR,R,E,D19_15um30cm_SC1-calib,12764,1770.185913,"[110.07184600830078, 113.80606079101562, 125.6...","[614.2388305664062, 218.9332733154297, 259.762...",17581.164062


In [15]:
for file_name in list_of_filenames_to_parse:
    
    # PSM
    psm_path = psm_files.get(file_name)

    psm = pd.read_csv(psm_path, sep = '\t', dtype={'Base Sequence': str, 'Missed Cleavages': str, 'Peptide Monoisotopic Mass':str, 'Mass Diff (Da)':str,'Mass Diff (ppm)':str,'	Protein Accession':str,'Peptide Description':str, 'Notch':str, 'Num Variable Mods':str, 'Decoy': str})

    #drop columns that we don't use
    psm = psm[['Matched Ion Series', 'Matched Ion Mass-To-Charge Ratios', 'Matched Ion Intensities', 'Matched Ion Counts', 'QValue', 'Full Sequence', 'Previous Amino Acid', 'Next Amino Acid', 'File Name', 'Scan Number']]
    psm = psm[psm['QValue'] < 0.01]

    psm = psm.rename({"Scan Number": "scan", "Full Sequence": "peptide"}, axis=1)

    #remove duplicate scans
    psm = psm.sort_values("QValue")
    psm = psm.drop_duplicates(subset=["scan"], keep="first")
    #make sure dtypes are correct
    psm[["scan"]] = psm[["scan"]].apply(pd.to_numeric)

    # MZML
    mzml_path = mzml_files.get(file_name)

    mzml = pyteomics.mzml.MzML(mzml_path)
    psm['mzml_info'] = psm.apply(lambda row: _mzml_helper(row, mzml),axis=1)
    psm[['Base Peak Intensity', 'mz_array', 'intensity_array', 'precursor_intenisty']] = pd.DataFrame(psm['mzml_info'].tolist(), index= psm.index)
    psm=psm.drop(columns='mzml_info')
    
    # Save changes as new file
    write_file_path = "data/parsed_psm/" + file_name + ".tsv"
    psm.to_csv(write_file_path, sep="\t")
    